In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import matplotlib
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
import os


In [12]:
def svd_decomposition_per_label(feature_space, feature, label):
   a = []
   for element in feature_space:
      if element['class'] == label:
         a.append(element[feature])
         
   a_array = np.array(a)

   U, S, Vh = np.linalg.svd(a_array, full_matrices=False, compute_uv=True, hermitian=False)

   return U, S, Vh
   


In [ ]:
# U, S, Vh = svd_decomposition(feature_space, feature)
# print(f"U.shape: {U.shape}, S.shape: {S.shape}, Vh.shape: {Vh.shape}")

#latent_feature_space = torch.load("/workspaces/dbm25/data/svd_latent_features.pt")

feature_space = torch.load("/workspaces/dbm25/data/extracted_features.pt")
label = "brain_glioma"
feature = "hog"

U, S, Vh = svd_decomposition_per_label(feature_space, feature, label)

print(f"U.shape: {U.shape}")



matrix_rank = np.linalg.matrix_rank(U)

num_images= U.shape[0]

latent_sum = np.sum(U, axis=0)       
latent_mean_glioma = latent_sum / num_images 

#print(f"Latent mean vector:\n{latent_mean}")


U.shape: (1002, 900)
Matrix rank (inherent dimensionality): 900


In [ ]:

label = "brain_menin"
feature = "hog"

U, S, Vh = svd_decomposition_per_label(feature_space, feature, label)

print(f"U.shape: {U.shape}")

num_images= U.shape[0]

matrix_rank = np.linalg.matrix_rank(U)

latent_sum = np.sum(U, axis=0)       
latent_mean_menin = latent_sum / num_images 

#print(f"Latent mean vector:\n{latent_mean}")


U.shape: (1002, 900)


In [ ]:

label = "brain_tumor"
feature = "hog"

U, S, Vh = svd_decomposition_per_label(feature_space, feature, label)

print(f"U.shape: {U.shape}")

num_images= U.shape[0]

matrix_rank = np.linalg.matrix_rank(U)

latent_sum = np.sum(U, axis=0)       
latent_mean_tumor = latent_sum / num_images 

#print(f"Latent mean vector:\n{latent_mean}")


U.shape: (1002, 900)


In [ ]:
#PARTE 2 DELLA TASK_7


label_latent_means = {
    "brain_glioma": latent_mean_glioma,
    "brain_menin": latent_mean_menin,
    "brain_tumor": latent_mean_tumor,
}

def predict_label_for_feature(query_feature, class_centroids, measure="cosine"):
    results = []
    for class_label, centroid in class_centroids.items():
        if measure == "cosine":
            score = cosine_similarity(query_feature.reshape(1, -1), centroid.reshape(1, -1))[0][0]
        else:  # Euclidean distance
            score = -np.linalg.norm(query_feature - centroid)
        results.append((class_label, score))
    results.sort(key=lambda x: x[1], reverse=True)
    return results[0][0] 



feature_space_part2 = torch.load("/workspaces/dbm25/data/svd_latent_features.pt")
feature = "svd_latent"


y_true = []
y_pred = []

for item in feature_space_part2:
    feat = np.array(item[feature])
    true_label = item["class"]

    pred_label = predict_label_for_feature(feat, label_latent_means, measure="cosine")

    y_true.append(true_label)
    y_pred.append(pred_label)

# Calcola precision, recall, f1 e accuracy
labels = list(label_latent_means.keys())
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, labels=labels, zero_division=0)
accuracy = accuracy_score(y_true, y_pred)

# Stampa metriche per ogni label
for i, label in enumerate(labels):
    print(f"Label: {label}")
    print(f"  Precision: {precision[i]:.4f}")
    print(f"  Recall:    {recall[i]:.4f}")
    print(f"  F1-score:  {f1[i]:.4f}")
    print()

print(f"Overall accuracy: {accuracy:.4f}")


KeyError: 'hog'